In [1]:
import os
import itertools as it
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import h5py 
import os
import itertools
import re
import functools
import statistics
import operator 
from sklearn.preprocessing import scale
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline
path = "/Users/mikebrennan/desktop/MillionSongSubset/data"
#path = "c:/users/brennm3/desktop/millionsongsubset/millionsongsubset/data"
save_load_path = '/Users/mikebrennan/Dropbox/MA755 Public (3)/pynotes/Michael-Nick'

#save_load_path = 'c:/Users/brennm3/Dropbox/Bentley/MA755/MA 755 Team Folder'
mss_df=pd.read_pickle(save_load_path+'/team_data_set.pkl')
to_cluster=mss_df[list(mss_df.columns[21:36]) + list(mss_df.columns[37:39]) + list(mss_df.columns[53:])]


height has been deprecated.



# Goal:


 + Use our data frame to detect similar songs using distance measures
 
### Let's try Euclidian Distance

In [2]:
def euc_dist(x,y):
    return(np.sqrt(np.sum(np.power((x-y),2))))

The DataFrame contains 9,552 songs and 43 measures.

In [3]:
print(to_cluster.shape)
to_cluster.head()

(9552, 43)


,tim_comp_0,tim_comp_1,tim_comp_2,tim_comp_3,tim_comp_4,tim_comp_5,tim_comp_6,tim_comp_7,tim_comp_8,tim_comp_9,tim_comp_10,tim_comp_11,tim_comp_12,tim_comp_13,tim_comp_14,pitch_length,max_segment,s1,f1,s2,f2,s3,f3,repeat_score,loudness_diff,vol_s1,vol_f1,vol_s2,vol_f2,vol_s3,vol_f3,c1_var,c2_var,c3_var,c4_var,c5_var,c6_var,c7_var,c8_var,c9_var,c10_var,c11_var,c12_var
0,-567.682577,147.664203,-339.192207,-554.155393,-43.886136,-503.389146,-101.305289,-205.727265,60.576452,-33.136826,7.785124,63.447632,-10.015682,-14.503595,33.596925,971,0.78041,0.260921,4.204082,0.203081,2.607595,0.157180,34.333333,1.770378,-0.353421,1.408664,242.000000,0.612883,4.864322,0.604046,6.453333,0.045620,0.063402,0.045679,0.036855,0.096425,0.027719,0.026924,0.027746,0.103268,0.051895,0.035965,0.070962
1,-426.597316,-403.199195,-337.360209,-41.048973,-20.271638,-34.718046,-38.285278,93.569841,87.339023,-47.498931,21.051709,-94.022645,80.848612,82.302616,-41.181983,550,0.54127,0.691640,3.722222,0.259435,33.500000,0.234835,67.000000,1.482162,-1.067109,1.328457,109.600000,1.313144,16.606061,0.442925,10.538462,0.015116,0.043025,0.085240,0.012999,0.059987,0.013305,0.023651,0.003128,0.011939,0.092856,0.007192,0.058272
2,-64.802953,-1146.409999,-600.498833,215.668447,36.408786,183.837237,13.173252,87.954370,22.416583,39.733235,-2.457589,-35.811513,63.243426,70.074740,-17.469580,562,2.43850,0.121708,2.352941,0.100500,22.857143,0.065489,4.571429,1.795608,-0.653810,0.495874,2.413793,0.471537,40.000000,0.412664,15.555556,0.075382,0.041664,0.024358,0.132694,0.014635,0.034790,0.015217,0.049511,0.016541,0.082187,0.108719,0.024884
3,-922.796743,-651.817408,324.914535,49.357092,-36.502033,50.429332,36.231150,-42.645043,2.982978,166.385512,-7.049662,62.155778,39.160533,69.148130,-58.329949,821,2.39759,0.127094,3.728814,0.115121,55.000000,0.063295,2.056075,1.705489,-1.874305,2.031918,163.600000,0.561950,14.607143,0.347581,8.432990,0.070540,0.006560,0.019793,0.018182,0.077537,0.102809,0.037265,0.070951,0.014226,0.038466,0.009483,0.050255
4,-744.210501,-333.304990,-65.009973,-347.351662,-322.512785,91.695531,127.889116,42.358802,10.011409,-88.682259,-78.130060,-55.528248,45.142637,-106.970837,23.810118,673,1.85115,0.120838,2.157303,0.074784,3.692308,0.060143,7.111111,1.644818,0.334646,0.412138,223.333333,0.401450,4.785714,0.390005,4.036145,0.018263,0.047061,0.021822,0.009878,0.030768,0.046678,0.074054,0.042758,0.030086,0.043015,0.093017,0.008699


### Step 1:  Normalize the data

In [4]:
scaled_data= scale(to_cluster, with_mean=True,with_std=True,axis=0)

### Step 2:  Measure "Distance" from target song to all songs

In [5]:
def song_distance(song_id):
    target=mss_df[mss_df['song_id']==song_id].index
    results=[]
    for i in range(0,len(scaled_data)):
        distance=euc_dist(scaled_data[target],scaled_data[i])
        results.append([mss_df.iloc[i]['song_id'],distance])

    euc_distance=pd.DataFrame(results,columns=['song_id','distance'])
    euc_distance=euc_distance.sort(['distance'])
    euc_distance=pd.merge(euc_distance,mss_df, on=['song_id'])
    return(euc_distance)

In [6]:
euc_distance = song_distance(3213)

/Users/mikebrennan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


### Step 3: Look at the top 5 matches.
 + Song = "The Space Between" 
 + Artist = Dave Matthews Band

In [7]:
euc_distance.head(6)

,song_id,distance,artist_familiarity,artist_hotttnesss,song_hotttnesss,title,artist_name,artist_location,release,artist_longitude,artist_latitude,duration,key,loudness,mode,tempo,time_signature,end_of_fade_in,start_of_fade_in,year,at_0,at_1,at_2,tim_comp_0,tim_comp_1,tim_comp_2,tim_comp_3,tim_comp_4,tim_comp_5,tim_comp_6,tim_comp_7,tim_comp_8,tim_comp_9,tim_comp_10,tim_comp_11,tim_comp_12,tim_comp_13,tim_comp_14,pitch_length,max_segment,mean_segment,key1,key2,key3,key4,key5,key6,key7,key8,key9,key10,key11,key12,song_id_2,s1,f1,s2,f2,s3,f3,repeat_score,loudness_diff,vol_s1,vol_f1,vol_s2,vol_f2,vol_s3,vol_f3,c1_var,c2_var,c3_var,c4_var,c5_var,c6_var,c7_var,c8_var,c9_var,c10_var,c11_var,c12_var
0,3213,0.000000,0.811890,0.657584,0.825233,b'The Space Between',b'DAVE MATTHEWS BAND',"b'Charlottesville, VA'",b'The Gorge',NaN,NaN,293.66812,6,-5.210,0,85.141,4,2.351,NaN,2001,b'jam band',b'alternative rock',b'rock',-805.595157,-555.578688,20.302660,314.101044,454.328362,-145.956953,91.514716,130.537292,-52.785766,166.632294,-63.617086,101.721162,-104.492805,-40.239942,-14.081365,781,10.91301,0.371233,0,0,0,0,0,1,0,0,0,0,0,0,3213,0.277537,11.040000,0.110053,69.0,0.074283,25.090909,1.867740,-1.735579,0.732236,259.333333,0.619528,33.826087,0.421106,5.440559,0.033546,0.054335,0.062330,0.031132,0.092967,0.015678,0.094469,0.047011,0.067748,0.076755,0.016756,0.023918
1,119,4.848458,0.652368,0.437965,0.394395,b'Niev nietch nievitch',b'Les Ogres De Barback',b'',b'Irfan_ le h\xc3\xa9ros',NaN,NaN,254.06649,4,-6.010,0,137.888,1,0.430,NaN,1999,b'chanson',b'gypsy jazz',b'chill-out',-860.121868,-932.546692,381.409332,168.652431,154.238869,75.125103,-5.628881,55.696803,-35.585967,193.487286,-0.764222,116.984641,38.505940,-103.249076,29.667383,1249,1.61800,0.200827,0,0,0,1,0,0,0,0,0,0,0,0,119,0.227553,6.800000,0.162716,59.5,0.111400,3.400000,1.686649,2.853325,0.771560,311.500000,0.525816,13.115789,0.510965,8.197368,0.026495,0.027334,0.036312,0.025305,0.056567,0.026960,0.061550,0.051621,0.031732,0.066992,0.010709,0.033861
2,5903,4.947662,0.687430,0.440715,0.434838,b'Let It All Go',b'Krayzie Bone',b'',b'Lyrical Paraphernalia',NaN,NaN,175.54240,6,-4.982,1,181.030,4,0.000,NaN,0,b'gangster rap',b'hip hop',b'rap',-604.960751,-93.774708,-330.350068,126.669247,174.028947,-43.032614,86.282731,99.401429,87.132548,126.289789,-8.062946,69.075413,-43.635604,-99.753504,1.310012,814,2.29877,0.214847,0,0,0,0,0,1,0,0,0,0,0,0,5903,0.210412,5.225806,0.184613,40.5,0.116187,2.612903,1.796025,-0.181385,1.171669,203.000000,0.593086,28.000000,0.445030,3.314286,0.058502,0.058132,0.036285,0.042875,0.028684,0.026745,0.064432,0.039967,0.080351,0.040625,0.053568,0.068970
3,4376,5.013931,0.872326,0.535891,0.253835,b'Better Believe It Remix (feat. Bun B_ Trae_ ...,b'Lil Boosie feat. Bun B_ Trae_ Yo Gotti & Foxx',b'',b'Thug Passion',NaN,NaN,318.79791,9,-13.135,1,125.827,7,0.000,NaN,2009,b'gangster rap',b'hip hop',b'rap',-923.120581,-571.861701,384.646739,212.943713,300.973873,-29.845707,150.746107,1.285663,-73.702257,0.624730,-4.059783,-86.072906,-43.410100,-29.207476,46.571279,1269,2.22639,0.249782,0,0,0,0,0,0,0,0,1,0,0,0,4376,0.329937,2.082192,0.243776,76.0,0.182672,7.414634,1.783786,-0.262852,0.857478,253.200000,0.413038,8.063694,0.302976,10.377049,0.051217,0.039563,0.062323,0.058004,0.058254,0.054890,0.069425,0.047537,0.030143,0.055681,0.019891,0.041721
4,2388,5.073339,0.545348,0.349001,0.334707,"b""Love Ain't A Play Thing""",b'Earl Hooker',"b'Clarksdale, MS'",b'Two Bugs And A Roach',-90.5651,34.19451,298.91873,2,-14.476,1,94.100,3,0.618,NaN,1968,b'chicago blues',b'delta blues',b'blues-rock',-338.596264,-489.731366,-268.680032,339.151580,176.213088,147.867379,143.143210,220.450775,-18.181186,176.045947,-39.336097,-13.906439,-76.891103,-55.171297,2.819215,1020,1.84140,0.289878,0,1,0,0,0,0,0,0,0,0,0,0,2388,0.320365,14.842105,0.148068,94.0,0.070476,4.947368,1.722413,0.288273,0.577443,339.333333,0.446059,18.509091,0.418422,2.367442,0.013731,0.071669,0.052548,0.019594,0.085183,0.008265,

### Look at Artist Terms of the top 100 matches.

In [8]:
euc_distance.head(100)['at_0'].value_counts().head()

b'hip hop'         10
b'chanson'          7
b'pop rock'         5
b'blues-rock'       4
b'gangster rap'     4
Name: at_0, dtype: int64

There are problems with these results.  The most common Artist Term in the top 100 results is "Hip Hop."  This does not align with the song/band at all.  The top 5 matches do not align wit the target song either.

## Attempt 2.  Use a Decision Tree with a training set.
Code for this section was pulled from the course text.

+ Most song recomendation engines require a person to first "like" or "dislike" songs.
+ A 99 song training set was created.  51 Likes 48 Dislikes.

In [9]:
from sklearn import tree
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.cross_validation import StratifiedShuffleSplit
import pprint

In [11]:
joy=pd.read_csv(save_load_path+'/taining_joy.csv')
scored=pd.concat([joy[joy['Like_Dislike']==1], joy[joy['Like_Dislike']==0]])
scored_data=pd.merge(mss_df,scored,on=['song_id'],how='inner')
print(scored_data.shape)
scored_data.head()

(99, 80)


,artist_familiarity,artist_hotttnesss,song_hotttnesss,title,artist_name,artist_location,release,artist_longitude,artist_latitude,duration,key,loudness,mode,tempo,time_signature,end_of_fade_in,start_of_fade_in,year,at_0,at_1,at_2,tim_comp_0,tim_comp_1,tim_comp_2,tim_comp_3,tim_comp_4,tim_comp_5,tim_comp_6,tim_comp_7,tim_comp_8,tim_comp_9,tim_comp_10,tim_comp_11,tim_comp_12,tim_comp_13,tim_comp_14,song_id,pitch_length,max_segment,mean_segment,key1,key2,key3,key4,key5,key6,key7,key8,key9,key10,key11,key12,song_id_2,s1,f1,s2,f2,s3,f3,repeat_score,loudness_diff,vol_s1,vol_f1,vol_s2,vol_f2,vol_s3,vol_f3,c1_var,c2_var,c3_var,c4_var,c5_var,c6_var,c7_var,c8_var,c9_var,c10_var,c11_var,c12_var,Like_Dislike
0,0.845769,0.788806,0.685940,b'Wicker Chair',b'Kings Of Leon',"b'Nashville, Tennessee'",b'Holy Roller Novocaine',-86.77836,36.16778,187.92444,0,-5.907,1,116.823,4,0.328,NaN,2003,b'southern rock',b'alternative rock',b'rock',-855.614094,55.022170,93.887443,-71.808226,-165.320962,113.499776,167.721608,39.326423,179.723055,-52.468952,-89.710739,48.582830,89.921551,1.727434,-76.769232,63,504,3.33864,0.368617,0,0,0,0,0,0,0,0,0,0,0,1,63,0.363289,15.636364,0.165110,3.909091,0.049925,2.023529,1.700322,5.412983,0.940640,83.666667,0.699593,5.837209,0.653878,8.508475,0.072295,0.051388,0.094597,0.009936,0.061203,0.019965,0.016988,0.020838,0.008057,0.007601,0.017022,0.109160,1
1,0.918452,0.819588,NaN,b'Spaceman',b'The Killers',b'',b'Hitzone 49',NaN,NaN,284.34240,4,-6.546,1,151.953,4,0.206,NaN,2008,b'rock',b'alternative',b'rave',-553.095179,41.041912,-333.806769,184.800496,40.255139,110.677080,-128.840652,-46.152777,-78.920389,-78.404440,31.408645,3.472324,74.970187,53.920906,-53.355195,225,1155,1.20154,0.244436,0,0,0,1,0,0,0,0,0,0,0,0,225,0.159226,54.000000,0.057687,6.136364,0.042603,2.076923,1.668178,-1.302113,2.285392,230.400000,0.338727,17.454545,0.259717,2.331984,0.012837,0.046787,0.008515,0.022123,0.061516,0.014917,0.016685,0.015706,0.039092,0.091766,0.043265,0.037010,1
2,0.834564,0.540216,NaN,b'Morning Glory',b'Oasis',"b'Manchester, England'",b'Stop The Clocks',NaN,NaN,301.92281,0,-3.159,1,136.948,4,5.619,NaN,0,b'british pop',b'hard rock',b'rap',-916.012134,-615.111723,126.289899,-515.775946,-97.857530,-371.576848,-339.250488,-507.190175,161.025931,-262.716809,259.228795,-148.361634,-87.769495,120.433483,6.219895,366,533,3.79719,0.567071,0,0,0,0,0,0,0,0,0,0,0,1,366,0.244787,57.600000,0.068551,7.384615,0.058199,2.285714,1.563931,2.607485,3.102771,132.500000,0.425229,14.722222,0.247511,7.464789,0.015514,0.007059,0.029219,0.006712,0.019148,0.007113,0.006606,0.004222,0.002513,0.005739,0.004614,0.048790,1
3,0.830885,0.776168,0.784712,"b""Don't Let Go""",b'Weezer',b'',b'The Green Album',NaN,NaN,180.97587,2,-3.637,1,142.680,4,2.635,NaN,2001,b'pop rock',b'rock',b'pop',-926.336965,-285.338426,68.642098,11.061597,-47.196346,60.328943,18.603784,-46.750733,-74.248155,-90.180067,-22.162657,-15.186563,5.086205,77.497868,-7.528511,442,477,2.71034,0.365697,0,1,0,0,0,0,0,0,0,0,0,0,442,0.307266,32.400000,0.239497,7.043478,0.115958,3.306122,1.594133,1.095542,0.408954,59.250000,0.376646,16.928571,0.334492,9.294118,0.006719,0.020304,0.044429,0.020598,0.055911,0.011597,0.055706,0.116721,0.027679,0.072987,0.025750,0.014161,1
4,0.748321,0.545866,0.000000,b'Ven Conmigo/Perdoname (Live)',b'Selena',b'Texas',b'La Leyenda',-100.07715,31.16890,388.51873,9,-5.680,0,106.438,1,0.195,NaN,0,b'tejano',b'cumbia',b'latin',-1223.968210,255.099951,417.745514,328.747050,140.310336,203.458365,201.602049,122.686575,119.930910,188.317513,-165.542480,145.741842,39.326486,-115.254477,37.471076,469,1394,3.92463,0.276683,0,0,0,0,0,0,0,0,1,0,0,0,469,0.172749,93.000000,0.043183,9.538462,0.035812,2.021739,1.739469,1.189635,0.341107,232.000000,0.268413,13.384615,0.251849,2.234350,0.055185,0.069767,0.017999,0.033748,0.025594,0.040422,0.027235,0.041377,0.050580,0.018787,0.046714,0.032440,0


### Split into X (independent) and Y (dependent) objects
+ X = Numerical Values about the song in the DataFrame
+ Y = 1/0 Binary Target.  1 = Like the song.  0 = Dislike the Song.

In [12]:
x=scored_data[list(scored_data.columns[21:36]) + list(scored_data.columns[37:39]) + list(scored_data.columns[53:79])]
y=scored_data.iloc[:,79:80]

### Our Functions (from the text)
+ get_train_test - Splits data into Training / Test using 80/20 Split
+ build_model - Builds our decision tree. 
+ test_model - Tests Model for accuracy.

In [13]:

def get_train_test(x,y):
    train_size=0.8
    test_size=1-train_size
    input_dataset=np.column_stack([x,y])
    stratified_split=StratifiedShuffleSplit(input_dataset[:,-1], test_size=test_size,n_iter=1,random_state=77)
    for train_indx,test_indx in stratified_split:
        train_x=input_dataset[train_indx,:-1]
        train_y=input_dataset[train_indx,-1]
        test_x=input_dataset[test_indx,:-1]
        test_y=input_dataset[test_indx,-1]
    return train_x,train_y,test_x,test_y

def build_model(x,y):
    model = tree.DecisionTreeClassifier(criterion="entropy",min_samples_leaf=20)
    model = model.fit(x,y)
    return model

def test_model(x,y,model):
    y_predicted = model.predict(x)
    print ("Model Accuracy = %0.2f"%(accuracy_score(y,y_predicted)*100)+"%\n")
    print( "\nConfusion Matrix")
    print ("===============")
    print (pprint.pprint(confusion_matrix(y,y_predicted)))
    print ("\nClassification Report")
    print ("===============")
    print (classification_report(y,y_predicted))
    
features=list(x.columns.values)

### Step 1: Split into Test and Training sets.

In [14]:
train_x,train_y,test_x,test_y = get_train_test(x,y)

### Step 2: Train the Model

In [15]:
model=build_model(train_x,train_y)

### Step 3: Evaluate the Results (Training Set)
+ 69.66% Accuracy
+ If Dislike: Predict Dislike 63% of the time
+ If Like: Predict Like 76% of the time. 

In [16]:
test_model(train_x,train_y,model)

Model Accuracy = 69.62%


Confusion Matrix
array([[24, 14],
       [10, 31]])
None

Classification Report
             precision    recall  f1-score   support

        0.0       0.71      0.63      0.67        38
        1.0       0.69      0.76      0.72        41

avg / total       0.70      0.70      0.69        79



### Step 4: Evaluate the Results (Test Set)
+ 75% Accuracy
+ If Dislike: Predict Dislike 70% of the time
+ If Like: Predict Like 80% of the time. 

In [17]:
test_model(test_x,test_y,model)

Model Accuracy = 75.00%


Confusion Matrix
array([[7, 3],
       [2, 8]])
None

Classification Report
             precision    recall  f1-score   support

        0.0       0.78      0.70      0.74        10
        1.0       0.73      0.80      0.76        10

avg / total       0.75      0.75      0.75        20



### Step 5: Evaluate the Tree
+ Identify the most important predictors
+ Visually examine the tree

In [18]:
importance=model.feature_importances_
for i, feature_name in enumerate(features):
    if importance[i]>0:
        print(feature_name, importance[i])

tim_comp_9 0.397220889663
max_segment 0.602779110337


In [19]:
tree.export_graphviz(model,out_file='tree.dot')

### Attempt the apply the model to the entire data set.
+ This will predict a 1/0 value to the remaining 9,453 un-ranked songs.

In [20]:
scoring=mss_df[~mss_df['song_id'].isin(list(scored['song_id']))]
scoring_set=scoring[list(scoring.columns[21:36]) + list(scoring.columns[37:39]) + list(scoring.columns[53:79])]
print(scoring_set.shape)


(9453, 43)


In [21]:
scoring['predict']=model.predict(scoring_set)


/Users/mikebrennan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### The top 10 Artist's for a "Like" Prediction

In [22]:
scoring[scoring['predict']==1]['artist_name'].value_counts().head(10)

b'Nick Cave & The Bad Seeds'    10
b'Bill & Gloria Gaither'         9
b'Les Ogres De Barback'          8
b'Aerosmith'                     8
b'Bon Jovi'                      8
b'Anti-Flag'                     7
b'Jeff And Sheri Easter'         7
b'UFO'                           7
b'Jacques Dutronc'               7
b'Dropkick Murphys'              7
Name: artist_name, dtype: int64

### The top 10 Artist's for a "Dislike" Prediction

In [23]:
scoring[scoring['predict']==0]['artist_name'].value_counts().head(10)

b'Sugar Minott'                 12
b'Phil Collins'                 10
b'RUN-DMC'                       8
b'Michael Jackson'               8
b'The Jackson Southernaires'     8
b'Jimmy Riley'                   8
b'B\xc3\xa9zu'                   8
b'Mighty Diamonds'               7
b'Tyrone Taylor'                 7
b'Helen Love'                    7
Name: artist_name, dtype: int64

### The top 5 Artist Terms for a "Like" Prediction

In [24]:
scoring[scoring['predict']==1]['at_0'].value_counts().head(5)

b'blues-rock'      191
b'ccm'             174
b'chanson'         113
b'post-grunge'      98
b'country rock'     76
Name: at_0, dtype: int64

### The top 5 Artist Terms for a "Dislike" Prediction

In [120]:
scoring[scoring['predict']==0]['at_0'].value_counts().head(5)

b'hip hop'         253
b'blues-rock'      146
b'roots reggae'    109
b'gangster rap'    108
b'dancehall'        88
Name: at_0, dtype: int64